In [1]:
!pip install keyboard pycaw

In [2]:
import keyboard
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

def get_volume_interface():
    """Gets the main audio endpoint volume interface."""
    devices = AudioUtilities.GetSpeakers()
    # We don't need to Activate or cast,
    # just access the .EndpointVolume property
    return devices.EndpointVolume

def main():
    print("Program started. Press 'Up Arrow' to increase volume, 'Down Arrow' to decrease.")
    print("Press 'Esc' to exit.")

    # Get the volume controller
    volume = get_volume_interface()
    
    # Define how much to change the volume by (0.05 = 5%)
    volume_step = 0.05

    while True:
        try:
            # Wait for the next key press event
            event = keyboard.read_event()

            # Check if the event is a key *press* (not a release)
            if event.event_type == keyboard.KEY_DOWN:
                
                # Get the current volume (0.0 to 1.0)
                current_volume = volume.GetMasterVolumeLevelScalar()

                if event.name == 'up':
                    # Calculate new volume, but don't go above 1.0 (100%)
                    new_volume = min(1.0, current_volume + volume_step)
                    volume.SetMasterVolumeLevelScalar(new_volume, None)
                    print(f"Volume Up: {new_volume * 100:.0f}%")

                elif event.name == 'down':
                    # Calculate new volume, but don't go below 0.0 (0%)
                    new_volume = max(0.0, current_volume - volume_step)
                    volume.SetMasterVolumeLevelScalar(new_volume, None)
                    print(f"Volume Down: {new_volume * 100:.0f}%")

                elif event.name == 'esc':
                    print("Exiting program.")
                    break  # Exit the while loop
        
        except Exception as e:
            print(f"An error occurred: {e}")
            break

if __name__ == "__main__":
    main()

Program started. Press 'Up Arrow' to increase volume, 'Down Arrow' to decrease.
Press 'Esc' to exit.
Volume Down: 74%
Volume Down: 69%
Volume Down: 64%
Volume Down: 59%
Volume Down: 54%
Volume Down: 49%
Volume Down: 44%
Volume Down: 39%
Volume Down: 34%
Volume Down: 29%
Volume Down: 24%
Volume Down: 19%
Volume Down: 14%
Volume Down: 9%
Volume Down: 4%
Volume Down: 0%
Volume Down: 0%
Volume Up: 5%
Volume Up: 10%
Volume Up: 15%
Volume Up: 20%
Volume Up: 25%
Volume Up: 30%
Volume Up: 35%
Volume Up: 40%
Volume Up: 45%
Volume Up: 50%
Volume Up: 55%
Volume Up: 60%
Volume Up: 65%
Volume Down: 60%
Volume Down: 55%
Volume Down: 50%
Volume Down: 45%
Volume Down: 40%
Volume Down: 35%
Volume Down: 30%
Volume Down: 25%
Volume Down: 20%
Volume Down: 15%
Volume Up: 20%
Volume Up: 25%
Volume Up: 30%
Volume Up: 35%
Volume Up: 40%
Volume Up: 45%
Volume Up: 50%
Volume Up: 55%
Volume Down: 50%
Volume Down: 45%
Volume Down: 40%
Volume Down: 35%
Volume Down: 30%
Volume Down: 25%
Volume Down: 20%
Volume Down

In [ ]:
import tkinter as tk
from tkinter import ttk
from comtypes import CLSCTX_ALL, CoCreateInstance, cast
from ctypes import POINTER
from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
from pycaw.constants import CLSID_MMDeviceEnumerator
import sys
import tkinter.font as tkfont

# Ensure Windows
if sys.platform != "win32":
    raise RuntimeError("This script requires Windows and pycaw (Windows Core Audio).")

# --- Audio setup (pycaw) ---
device_enumerator = CoCreateInstance(CLSID_MMDeviceEnumerator, IMMDeviceEnumerator, clsctx=CLSCTX_ALL)
device = device_enumerator.GetDefaultAudioEndpoint(0, 1)  # eRender, eMultimedia
interface = device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

_ignore_slider_callback = False

def get_current_volume_percent():
    return int(round(volume.GetMasterVolumeLevelScalar() * 100.0))

def safe_cfg(widget, **kwargs):
    """Safely configure a widget if it still exists (prevents TclError during shutdown)."""
    try:
        if widget and widget.winfo_exists():
            widget.config(**kwargs)
    except Exception:
        # ignore GUI errors (widget might be destroyed)
        pass

def set_volume_percent(percent):
    global _ignore_slider_callback
    percent = max(0, min(100, int(round(percent))))
    _ignore_slider_callback = True
    volume.SetMasterVolumeLevelScalar(percent / 100.0, None)
    try:
        if 'volume_slider' in globals() and volume_slider.winfo_exists():
            volume_slider.set(percent)
        if 'volume_progress' in globals() and volume_progress.winfo_exists():
            volume_progress['value'] = percent
        if 'volume_value_label' in globals() and volume_value_label.winfo_exists():
            safe_cfg(volume_value_label, text=f"{percent}%")
        update_speaker_icon(percent)
    finally:
        root.after(10, lambda: set_ignore_false())

def set_ignore_false():
    global _ignore_slider_callback
    _ignore_slider_callback = False

def on_slider_change(val):
    global _ignore_slider_callback
    if _ignore_slider_callback:
        return
    try:
        v = float(val)
    except Exception:
        return
    volume.SetMasterVolumeLevelScalar(v / 100.0, None)
    # safely update UI parts (they might have been destroyed)
    try:
        if 'volume_progress' in globals() and volume_progress.winfo_exists():
            volume_progress['value'] = int(round(v))
        if 'volume_value_label' in globals() and volume_value_label.winfo_exists():
            safe_cfg(volume_value_label, text=f"{int(round(v))}%")
        update_mute_button_text()
        update_speaker_icon(int(round(v)))
    except Exception:
        pass

def toggle_mute():
    try:
        is_muted = bool(volume.GetMute())
        volume.SetMute(0 if is_muted else 1, None)
        update_mute_button_text()
        # if unmuted, ensure UI shows actual volume
        if not bool(volume.GetMute()):
            cur = get_current_volume_percent()
            try:
                if 'volume_progress' in globals() and volume_progress.winfo_exists():
                    volume_progress['value'] = cur
                if 'volume_value_label' in globals() and volume_value_label.winfo_exists():
                    safe_cfg(volume_value_label, text=f"{cur}%")
                if 'volume_slider' in globals() and volume_slider.winfo_exists():
                    volume_slider.set(cur)
            except Exception:
                pass
        update_speaker_icon(get_current_volume_percent())
    except Exception:
        pass

def update_mute_button_text():
    # Guard widget update to avoid TclError if widget was destroyed
    try:
        if 'mute_button' in globals() and mute_button.winfo_exists():
            current_mute = bool(volume.GetMute())
            mute_button.config(text="Unmute" if current_mute else "Mute")
    except Exception:
        pass

def volume_up(step=10):
    cur = get_current_volume_percent()
    set_volume_percent(min(cur + step, 100))

def volume_down(step=10):
    cur = get_current_volume_percent()
    set_volume_percent(max(cur - step, 0))

def on_key_press(event):
    key = event.keysym.lower()
    if key == 'i':
        volume_up()
    elif key == 'd':
        volume_down()
    elif key == 'm':
        toggle_mute()

def update_speaker_icon(percent):
    try:
        if 'speaker_label' in globals() and speaker_label.winfo_exists():
            if bool(volume.GetMute()):
                icon = "🔇"
            elif percent == 0:
                icon = "🔈"
            elif percent < 50:
                icon = "🔉"
            else:
                icon = "🔊"
            safe_cfg(speaker_label, text=icon)
    except Exception:
        pass

# --- Build UI (styled) ---
root = tk.Tk()
root.title("Keyboard Volume Controller — Styled (robust)")
root.geometry("420x260")
root.resizable(False, False)

# Global style
style = ttk.Style(root)
style.theme_use('clam')

# Colors
BG = "#0f1724"
CARD = "#0b1220"
ACCENT = "#0ea5e9"
TEXT = "#e6eef6"
SUBTEXT = "#9fb8ca"

root.configure(bg=BG)

# Fonts (valid weights only)
title_font = tkfont.Font(family="Segoe UI", size=14, weight="bold")
big_font = tkfont.Font(family="Segoe UI", size=22, weight="bold")
label_font = tkfont.Font(family="Segoe UI", size=10)
btn_font = tkfont.Font(family="Segoe UI", size=10, weight="bold")
value_font = tkfont.Font(family="Segoe UI", size=12, weight="bold")

# Card frame
card = tk.Frame(root, bg=CARD, bd=0, relief=tk.FLAT)
card.place(relx=0.03, rely=0.05, relwidth=0.94, relheight=0.9)

# Header row
header_frame = tk.Frame(card, bg=CARD)
header_frame.pack(fill="x", padx=16, pady=(12, 6))

title_label = tk.Label(header_frame, text="Volume Control", bg=CARD, fg=TEXT, font=title_font)
title_label.pack(side="left")

speaker_label = tk.Label(header_frame, text="🔊", bg=CARD, fg=ACCENT, font=big_font)
speaker_label.pack(side="right")

# Progress + value
progress_frame = tk.Frame(card, bg=CARD)
progress_frame.pack(fill="x", padx=16, pady=(6, 8))

volume_progress = ttk.Progressbar(progress_frame, orient="horizontal", mode="determinate", length=310)
volume_progress.pack(side="left", fill="x", expand=True, padx=(0,10), pady=4)

volume_value_label = tk.Label(progress_frame, text=f"{get_current_volume_percent()}%", bg=CARD, fg=TEXT, font=value_font)
volume_value_label.pack(side="right")

# Slider
slider_frame = tk.Frame(card, bg=CARD)
slider_frame.pack(fill="x", padx=12, pady=(0, 8))

style.configure("Horizontal.TScale", troughcolor="#111827", background=ACCENT)
volume_slider = ttk.Scale(slider_frame, from_=0, to=100, orient="horizontal", command=on_slider_change, style="Horizontal.TScale")
volume_slider.set(get_current_volume_percent())
volume_slider.pack(fill="x", padx=6, pady=6)

# Buttons
btn_frame = tk.Frame(card, bg=CARD)
btn_frame.pack(fill="x", padx=16, pady=(8, 12))

style.configure("Accent.TButton", font=btn_font, foreground="white", background=ACCENT, padding=(8,6))
style.map("Accent.TButton", background=[('active', '#06a0d8'), ('!disabled', ACCENT)])

style.configure("Outline.TButton", font=btn_font, foreground=TEXT, background="#1b2530", padding=(8,6))
style.map("Outline.TButton", background=[('active', '#19232b')])

mute_button = ttk.Button(btn_frame, text="Mute", command=toggle_mute, style="Outline.TButton")
mute_button.pack(side="left", padx=(0,8), ipadx=6, expand=True, fill="x")

vol_down_button = ttk.Button(btn_frame, text="-10%", command=volume_down, style="Outline.TButton")
vol_down_button.pack(side="left", padx=6, ipadx=6)

vol_up_button = ttk.Button(btn_frame, text="+10%", command=volume_up, style="Accent.TButton")
vol_up_button.pack(side="left", padx=(6,0), ipadx=6)

footer = tk.Label(card, text='Shortcuts: "i" increase, "d" decrease, "m" mute/unmute', bg=CARD, fg=SUBTEXT, font=label_font)
footer.pack(side="bottom", pady=(0,12))

root.bind_all("<Key>", on_key_press)

def poll_volume():
    try:
        cur = get_current_volume_percent()
        global _ignore_slider_callback
        if not _ignore_slider_callback:
            _ignore_slider_callback = True
            try:
                if volume_slider.winfo_exists():
                    volume_slider.set(cur)
                if volume_progress.winfo_exists():
                    volume_progress['value'] = cur
                if volume_value_label.winfo_exists():
                    safe_cfg(volume_value_label, text=f"{cur}%")
            except Exception:
                pass
            root.after(10, set_ignore_false)
        update_mute_button_text()
        update_speaker_icon(cur)
    except Exception as e:
        # device disconnected / other errors — ignore to keep UI alive
        # print("Poll warning:", e)
        pass
    finally:
        if root.winfo_exists():
            root.after(500, poll_volume)

# Initial setup
update_mute_button_text()
update_speaker_icon(get_current_volume_percent())
root.after(500, poll_volume)

root.mainloop()


In [ ]:
"""
mic_hotkeys_cominit_fixed.py
Global hotkeys to control microphone volume (Windows).
This version ensures COM is initialized inside each hotkey handler thread.

Hotkeys:
  Ctrl+Alt+Up    -> increase mic volume by 5%
  Ctrl+Alt+Down  -> decrease mic volume by 5%
  Ctrl+Alt+M     -> toggle mute/unmute
  Ctrl+Alt+Q     -> stop hotkeys (clean shutdown)
"""
import sys
import time
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

# 3rd-party libraries
try:
    import keyboard   # pip install keyboard
except Exception:
    raise SystemExit("Install required package: pip install keyboard")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# Constants
eCapture = 1
eConsole = 0
STEP_PERCENT = 5.0

# Stop event for clean shutdown
stop_event = threading.Event()

# Helper: create IMMDeviceEnumerator (typed)
def _create_mmdevice_enumerator():
    try:
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    """
    Returns an IAudioEndpointVolume pointer for the default capture device.
    Caller must ensure COM is initialized in the calling thread.
    """
    enumerator = _create_mmdevice_enumerator()
    default_device = enumerator.GetDefaultAudioEndpoint(eCapture, eConsole)
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    return cast(iface, POINTER(IAudioEndpointVolume))

def _percent_to_scalar(p):
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    return max(0.0, min(100.0, s * 100.0))

# Decorator to ensure COM initialized per-thread for hotkey handlers
def ensure_com(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            # Print traceback for easier debugging
            import traceback
            print("Exception in handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            # Uninitialize COM in this thread when handler completes
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

# Handlers (each will run with COM initialized)
@ensure_com
def increase_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = min(100.0, cur_pct + STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[+] Mic volume -> {new_pct:.0f}%")

@ensure_com
def decrease_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = max(0.0, cur_pct - STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[-] Mic volume -> {new_pct:.0f}%")

@ensure_com
def toggle_mute():
    vol = _get_volume_interface_for_default()
    cur_mute = bool(vol.GetMute())
    new_mute = not cur_mute
    vol.SetMute(1 if new_mute else 0, None)
    # Print the new state (M when muted, U when unmuted)
    print(f"[{'M' if new_mute else 'U'}] Mic muted -> {new_mute}")

def quit_program():
    """
    Signal the main loop to stop and unhook hotkeys.
    Avoids calling sys.exit() directly from a hotkey handler.
    """
    print("Stopping hotkeys (quit requested)...")
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass
    stop_event.set()

def register_hotkeys():
    # Avoid duplicate registrations if re-run
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass

    keyboard.add_hotkey('ctrl+alt+up', increase_volume)
    keyboard.add_hotkey('ctrl+alt+down', decrease_volume)
    keyboard.add_hotkey('ctrl+alt+m', toggle_mute)
    keyboard.add_hotkey('ctrl+alt+q', quit_program)

if __name__ == "__main__":
    print("Registering hotkeys...")
    register_hotkeys()

    # Try to print initial state (initialize COM briefly here)
    CoInitialize()
    try:
        try:
            vol = _get_volume_interface_for_default()
            print(
                f"Initial mic volume: {_scalar_to_percent(float(vol.GetMasterVolumeLevelScalar())):.0f}%"
                f"  muted={bool(vol.GetMute())}"
            )
        except Exception as e:
            print("Could not read initial mic state:", e)
    finally:
        CoUninitialize()

    print("Hotkeys active. Press Ctrl+Alt+Up/Down to change mic, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.")
    try:
        # Main loop: wait until quit_program() sets stop_event
        while not stop_event.wait(timeout=0.1):
            # keep the main thread responsive; checks stop_event every 0.1s
            pass
    except KeyboardInterrupt:
        print("Interrupted by user; exiting.")
    finally:
        # clean up in any case
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        print("Exited mic hotkeys.")


Registering hotkeys...
Initial mic volume: 69%  muted=False
Hotkeys active. Press Ctrl+Alt+Up/Down to change mic, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.
[+] Mic volume -> 74%
[-] Mic volume -> 69%
[-] Mic volume -> 64%
[+] Mic volume -> 74%
[-] Mic volume -> 69%
[+] Mic volume -> 74%


In [1]:
"""
mic_hotkeys_cominit_fixed.py
Global hotkeys to control microphone volume (Windows).
This version ensures COM is initialized inside each hotkey handler thread.

Hotkeys:
  Ctrl+Alt+Up    -> increase mic volume by 5%
  Ctrl+Alt+Down  -> decrease mic volume by 5%
  Ctrl+Alt+M     -> toggle mute/unmute
  Ctrl+Alt+Q     -> stop hotkeys (clean shutdown)
"""
import sys
import time
import platform
import threading
from ctypes import POINTER, cast
from comtypes import CLSCTX_ALL, CoInitialize, CoUninitialize
from comtypes.client import CreateObject
from comtypes import GUID
from functools import wraps

if platform.system() != "Windows":
    raise SystemExit("This script runs only on Windows.")

# 3rd-party libraries
try:
    import keyboard   # pip install keyboard
except Exception:
    raise SystemExit("Install required package: pip install keyboard")

try:
    from pycaw.pycaw import IAudioEndpointVolume, IMMDeviceEnumerator
except Exception:
    raise SystemExit("Install required packages: pip install pycaw comtypes")

# Constants
eCapture = 0   # <-- changed from 1 to 0 so the default endpoint is the render (output) device
eConsole = 0
STEP_PERCENT = 5.0

# Stop event for clean shutdown
stop_event = threading.Event()

# Helper: create IMMDeviceEnumerator (typed)
def _create_mmdevice_enumerator():
    try:
        return CreateObject("MMDeviceEnumerator.MMDeviceEnumerator", interface=IMMDeviceEnumerator)
    except Exception:
        clsid = GUID("{BCDE0395-E52F-467C-8E3D-C4579291692E}")
        return CreateObject(clsid, interface=IMMDeviceEnumerator)

def _get_volume_interface_for_default():
    """
    Returns an IAudioEndpointVolume pointer for the default capture device.
    Caller must ensure COM is initialized in the calling thread.
    """
    enumerator = _create_mmdevice_enumerator()
    default_device = enumerator.GetDefaultAudioEndpoint(eCapture, eConsole)
    iface = default_device.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    return cast(iface, POINTER(IAudioEndpointVolume))

def _percent_to_scalar(p):
    return max(0.0, min(1.0, p / 100.0))

def _scalar_to_percent(s):
    return max(0.0, min(100.0, s * 100.0))

# Decorator to ensure COM initialized per-thread for hotkey handlers
def ensure_com(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        CoInitialize()
        try:
            return func(*args, **kwargs)
        except Exception:
            # Print traceback for easier debugging
            import traceback
            print("Exception in handler:", file=sys.stderr)
            traceback.print_exc()
        finally:
            # Uninitialize COM in this thread when handler completes
            try:
                CoUninitialize()
            except Exception:
                pass
    return wrapper

# Handlers (each will run with COM initialized)
@ensure_com
def increase_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = min(100.0, cur_pct + STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[+] Mic volume -> {new_pct:.0f}%")

@ensure_com
def decrease_volume():
    vol = _get_volume_interface_for_default()
    cur = float(vol.GetMasterVolumeLevelScalar())
    cur_pct = _scalar_to_percent(cur)
    new_pct = max(0.0, cur_pct - STEP_PERCENT)
    vol.SetMasterVolumeLevelScalar(_percent_to_scalar(new_pct), None)
    print(f"[-] Mic volume -> {new_pct:.0f}%")

@ensure_com
def toggle_mute():
    vol = _get_volume_interface_for_default()
    cur_mute = bool(vol.GetMute())
    new_mute = not cur_mute
    vol.SetMute(1 if new_mute else 0, None)
    # Print the new state (M when muted, U when unmuted)
    print(f"[{'M' if new_mute else 'U'}] Mic muted -> {new_mute}")

def quit_program():
    """
    Signal the main loop to stop and unhook hotkeys.
    Avoids calling sys.exit() directly from a hotkey handler.
    """
    print("Stopping hotkeys (quit requested)...")
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass
    stop_event.set()

def register_hotkeys():
    # Avoid duplicate registrations if re-run
    try:
        keyboard.unhook_all_hotkeys()
    except Exception:
        pass

    keyboard.add_hotkey('ctrl+alt+up', increase_volume)
    keyboard.add_hotkey('ctrl+alt+down', decrease_volume)
    keyboard.add_hotkey('ctrl+alt+m', toggle_mute)
    keyboard.add_hotkey('ctrl+alt+q', quit_program)

if __name__ == "__main__":
    print("Registering hotkeys...")
    register_hotkeys()

    # Try to print initial state (initialize COM briefly here)
    CoInitialize()
    try:
        try:
            vol = _get_volume_interface_for_default()
            print(
                f"Initial mic volume: {_scalar_to_percent(float(vol.GetMasterVolumeLevelScalar())):.0f}%"
                f"  muted={bool(vol.GetMute())}"
            )
        except Exception as e:
            print("Could not read initial mic state:", e)
    finally:
        CoUninitialize()

    print("Hotkeys active. Press Ctrl+Alt+Up/Down to change mic, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.")
    try:
        # Main loop: wait until quit_program() sets stop_event
        while not stop_event.wait(timeout=0.1):
            # keep the main thread responsive; checks stop_event every 0.1s
            pass
    except KeyboardInterrupt:
        print("Interrupted by user; exiting.")
    finally:
        # clean up in any case
        try:
            keyboard.unhook_all_hotkeys()
        except Exception:
            pass
        print("Exited mic hotkeys.")


Registering hotkeys...
Initial mic volume: 63%  muted=False
Hotkeys active. Press Ctrl+Alt+Up/Down to change mic, Ctrl+Alt+M to toggle mute, Ctrl+Alt+Q to quit.
[+] Mic volume -> 68%
Stopping hotkeys (quit requested)...
Exited mic hotkeys.
